# Point Cloud Tutorial

In this notebook we will cover:
* Loading RGBD data and creating an [Open3D](http://www.open3d.org/) point cloud
* Removing points with inaccurate depth
* Cropping and downsampling a point cloud
* Getting the highest, lowest and random points.
* Project points from 3D to 2D



We will use data of a robot holding a shirt in the air, prerecorded with a ZED 2i.

> Download this data from [here](https://ugentbe-my.sharepoint.com/:f:/g/personal/victorlouis_degusseme_ugent_be/EkIZoyySsnZBg56hRq1BqdkBuGlvhAwPWT9HDuqaUB-psA?e=iSehj6) and save the folder in a folder called `data` relative to this notebook.


In [ ]:
%matplotlib inline

In [ ]:
import os
data_dir = os.path.join("data", "competition_sample_0000")

In [ ]:
!ls $data_dir

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

from airo_dataset_tools.data_parsers.camera_intrinsics import CameraIntrinsics
from airo_dataset_tools.data_parsers.pose import Pose

In [ ]:
intrinsics_path = os.path.join(data_dir, "intrinsics.json")
image_left_path = os.path.join(data_dir, "image_left.png")
image_right_path = os.path.join(data_dir, "image_right.png")
depth_map_path = os.path.join(data_dir, "depth_map.tiff")
confidence_map_path = os.path.join(data_dir, "confidence_map.tiff")
pointcloud_path = os.path.join(data_dir, "pointcloud.ply")
camera_pose_path = os.path.join(data_dir, "camera_pose.json")

## 1. Loading the data

### 1.1 Loading the camera parameters

In [ ]:
with open(intrinsics_path, "r") as f:
    intrinsics_model = CameraIntrinsics.model_validate_json(f.read())
    intrinsics = intrinsics_model.as_matrix()
    resolution = intrinsics_model.image_resolution.as_tuple()

with open(camera_pose_path, "r") as f:
    camera_pose = Pose.model_validate_json(f.read()).as_homogeneous_matrix()

with np.printoptions(precision=3, suppress=True):
    print("Resolution:", resolution)
    print("Intrinsics: \n", intrinsics)
    print("Extrinsics: \n", camera_pose)

### 1.2 Loading the color images

In [ ]:
image_left = plt.imread(image_left_path) # you can also use cv2.imread but then you get BGR instead of RGB
image_right = plt.imread(image_right_path)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(image_left)
plt.title("Left image")
plt.subplot(1, 2, 2)
plt.imshow(image_right)
plt.title("Right image")
plt.show()

### 1.3 Loading the depth and confidence map

> Note: the confidence map has range [0.0, 100.0] where 0.0 is the **most confident**.

In [ ]:
depth_map = cv2.imread(depth_map_path, cv2.IMREAD_ANYDEPTH)
confidence_map = cv2.imread(confidence_map_path, cv2.IMREAD_ANYDEPTH)

print("depth_map.dtype:", depth_map.dtype)
print("confidence_map.dtype:", confidence_map.dtype)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(depth_map)
plt.title("Depth map")
plt.colorbar(fraction=0.025, pad=0.04)
plt.subplot(1, 2, 2)
plt.imshow(confidence_map)
plt.title("Confidence map")
plt.colorbar(fraction=0.025, pad=0.04)
plt.show()

### 1.4 Loading the point cloud

Open3D uses the abbreviation `pcd` for their [PointCloud](http://www.open3d.org/docs/release/python_api/open3d.geometry.PointCloud.html#open3d.geometry.PointCloud) object, we will use this too to distinguish between them and numpy pointclouds.

In [ ]:
pcd_in_camera = o3d.io.read_point_cloud(pointcloud_path)
pcd = pcd_in_camera.transform(camera_pose) # transform to world frame (= base frame of left robot)

In [ ]:
o3d.visualization.draw_geometries([pcd])

### 1.5 Numpy point clouds

Open3D provides a lot functionality for point clouds, see their [tutorial](http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html).
However sometimes you need something custom, (e.g. getting the lowest and highest points).
This can be done easily by converting the Open3D point cloud to numpy arrays.

In [ ]:
points_float64 = np.asarray(pcd.points)
colors_float64 = np.asarray(pcd.colors)

print(f"width x height = {resolution[0]} x {resolution[1]} = {resolution[0] * resolution[1]}")
print("points:", points_float64.shape, points_float64.dtype)
print("colors:", colors_float64.shape, colors_float64.dtype)

> Note: the amount of points in the point cloud is equal to the amount of pixels in the images and depth map.

> Note: Open3D uses float64 for pointcloud positions and colors

In [ ]:
from airo_camera_toolkit.point_clouds.conversions import pointcloud_open3d_to_numpy

points, colors = pointcloud_open3d_to_numpy(pcd)

print("points:", points.shape, points.dtype)
print("colors:", colors.shape, colors.dtype)

> Note: converting large point clouds to and from Open3D is slow, so only do this when necessary. It also introduces some rounding error.

In [ ]:
%%timeit
points, colors = pointcloud_open3d_to_numpy(pcd)

In [ ]:
%%timeit
pcd = pointcloud_numpy_to_open3d((points, colors))

In [ ]:
from airo_camera_toolkit.point_clouds.conversions import pointcloud_numpy_to_open3d

pcd2 = pointcloud_numpy_to_open3d((points, colors))

print("Points exactly equal:", np.all(np.asarray(pcd.points) == np.asarray(pcd2.points)))
print("Colors exactly equal:", np.all(np.asarray(pcd.colors) == np.asarray(pcd2.colors)))
print("Points close:", np.allclose(np.asarray(pcd.points), np.asarray(pcd2.points)))
print("Colors close:", np.allclose(np.asarray(pcd.colors), np.asarray(pcd2.colors)))

In [ ]:
o3d.visualization.draw_geometries([pcd2])

## 2. Removing low confidence points

In [ ]:
from airo_typing import NumpyDepthMapType
from ipywidgets import interact

def binarize_confidence(image: NumpyDepthMapType, threshold=50.0):
    # confident = np.rint(image).astype(np.uint8) <= np.rint(threshold).astype(np.uint8)
    confident = image <= threshold
    return confident

@interact(threshold=(0.0, 100.0, 1.0))
def show_confidence_binarized(threshold=50.0):
    confidence_binarized = binarize_confidence(confidence_map, threshold)
    confidence_image = confidence_binarized.astype(np.uint8) * 255
    plt.figure(figsize=(10, 5))
    plt.imshow(confidence_image, vmin=0, vmax=255)
    plt.colorbar(fraction=0.025, pad=0.04)
    plt.show()

In [ ]:
threshold = 1.0  # a value of 1.0 means only the most confidence points will be kept
confidence_binarized = binarize_confidence(confidence_map, threshold)